In [1]:
## Austin Caudill
## 2-23-2023

## Install required Packages:
## !pip install simplekml

In [2]:
import os
import pandas as pd
import simplekml
import zipfile

from simplekml import Region, LatLonAltBox, Lod ## Not sure if all this is needed.

os.chdir("C:/Users/Austin/Dropbox (Daylight Petroleum)/Well Master")

# Load well data from CSV file
well_data = pd.read_excel('DLP Well Master.xlsx')

os.chdir("C:/Users/Austin/Documents/GitHub/Oil-and-Gas-KML-Generator")

# Create a KML object
kml = simplekml.Kml()

In [3]:
# Define icon URLs for different well types and statuses with Python Dictionaries.

icon_urls = {
    ('OIL WELL', 'ACTIVE'): 'Icons/Active/OIL WELL.png',
    ('OIL WELL', 'INACTIVE'): 'Icons/Inactive/OIL WELL.png',
    ('OIL WELL', 'TA'): 'Icons/Inactive/OIL WELL.png',
    ('OIL WELL', 'PA'): 'Icons/Plugged/Plugged-All.png',
    ('GAS WELL', 'ACTIVE'): 'Icons/Active/GAS WELL.png',
    ('GAS WELL', 'INACTIVE'): 'Icons/Inactive/GAS WELL.png',
    ('GAS WELL', 'TA'): 'Icons/Inactive/GAS WELL.png',
    ('GAS WELL', 'PA'): 'Icons/Plugged/Plugged-All.png',
    ('INJECTION WELL', 'ACTIVE'): 'Icons/Active/WATER.png',
    ('INJECTION WELL', 'INACTIVE'): 'Icons/Inactive/WATER.png',
    ('INJECTION WELL', 'TA'): 'Icons/Inactive/WATER.png',
    ('INJECTION WELL', 'PA'): 'Icons/Plugged/Plugged-All.png',
    ('SWD', 'ACTIVE'): 'Icons/Active/WATER.png',
    ('SWD', 'TA'): 'Icons/Inactive/WATER.png',    
    ('SWD', 'PA'): 'Icons/Plugged/Plugged-All.png',
    ('WSW', 'ACTIVE'): 'Icons/Active/WATER.png',
    ('WSW', 'INACTIVE'): 'Icons/Inactive/WATER.png',
    ('WSW', 'TA'): 'Icons/Inactive/WATER.png',
}


In [4]:
# Loop through each unique combination of District, Area, Route, and EngGroup, and add points to the KML object
for district, district_group in well_data.groupby('District'):
    district_folder = kml.newfolder(name=district)

    for area, area_group in district_group.groupby('Area'):
        area_folder = district_folder.newfolder(name=area)

        for route, route_group in area_group.groupby('Route'):
            route_folder = area_folder.newfolder(name=route)

            for enggroup, group in route_group.groupby('EngGroup'):
                enggroup_folder = route_folder.newfolder(name=enggroup)

                # Loop through each well in this group and add a point to the folder
                for index, row in group.iterrows():
                    # Get well name, coordinates, WellType, and Well_Status values
                    well_name = row['WellName']
                    lat = row['Lat']
                    lon = row['Long']
                    well_type = row['WellType']
                    well_status = row['WELL_STATUS']
                    coords = [(lon, lat)]

                    # Create a new point with a custom icon style
                    point = enggroup_folder.newpoint(name=well_name, coords=coords)

                    # Set the icon image for the point based on the WellType and Well_Status values
                    icon_url = icon_urls.get((well_type, well_status), None)
                    if icon_url is not None:
                        icon = simplekml.Icon(href=icon_url)
                        point.style.iconstyle.icon = icon
                    else:
                        icon_url = "Icons/Unknown.png"
                        icon = simplekml.Icon(href=icon_url)
                        point.style.iconstyle.icon = icon
                        
                    # Ensures highlight icon is same from if statement above.
                    highlight_icon = icon

                    # Set a highlight style for the point to show the label on hover.
                    highlight_style = simplekml.Style()
                    highlight_style.labelstyle.color = 'ffffffff'
                    highlight_style.labelstyle.scale = 1.5
                    highlight_style.labelstyle.visibility = 1
                    highlight_style.iconstyle.scale = 1.2
                    highlight_style.iconstyle.icon = highlight_icon
                    
                    point.stylemap.normalstyle = point.style
                    point.stylemap.highlightstyle = highlight_style
                    
                    # Set the label style for the point
                    point.style.labelstyle.scale = 0 # To hide initially
                    point.viewrefreshmode = simplekml.ViewRefreshMode.onstop
                    point.viewrefreshtime = 1 # seconds
                    point.viewboundscale = 1
                        
                    # Create a description for the point
                    description = f'<b>API:</b> {row["API"]}<br>' \
                                    f'<b>District:</b> {row["District"]}<br>' \
                                    f'<b>Area:</b> {row["Area"]}<br>' \
                                    f'<b>Acq:</b> {row["DP_Acq"]}<br>' \
                                    f'<b>Route:</b> {row["Route"]}<br>' \
                                    f'<b>Eng Group:</b> {row["EngGroup"]}<br>' \
                                    f'<b>County:</b> {row["County"]}<br>' \
                                    f'<b>State:</b> {row["STATE"]}<br>' \
                                    f'<b>AL Type:</b> {row["AL_Type"]}<br>' \
                                    f'<b>Production Engineer:</b> {row["ProdEngineer"]}<br>'

                    point.description = description                                                      

In [5]:
# Save the KML file
kml.save('well_data.kml')

In [6]:
# Define the path to the KML file
kml_path = 'well_data.kml'

# Save the KML file
kml.save(kml_path)

print("Finished.")

Finished.


In [7]:
# Let's create a Network Linked KML!
kml = simplekml.Kml()

networklink = kml.newnetworklink(name='DLP KML')
link = networklink.link
link.href = 'https://raw.githubusercontent.com/AustinCaudill/Oil-and-Gas-KML-Generator/main/well_data.kml'
link.viewrefreshmode = simplekml.ViewRefreshMode.onrequest

kml.save('networklink.kml')


print("Finished.")

Finished.
